# **1. Import Required Libraries**

In [1]:
#!pip install pdfplumber

In [2]:
#pip install python-docx

In [3]:
# folder_path = "Datasets/data" # Path of the resume

## **Here's the complete list of required libraries with installation commands:**

In [4]:
# # Core Requirements
# pip install pandas numpy python-dateutil

# # File Processing
# pip install pdfplumber python-docx

# # NLP Processing
# pip install spacy gensim scikit-learn

# # Advanced Text Processing
# pip install nltk regex

# # Create requirements.txt file:
# echo "pandas>=1.3.5
# numpy>=1.21.6
# pdfplumber>=0.7.4
# python-docx>=0.8.11
# spacy>=3.4.1
# gensim>=4.2.0
# scikit-learn>=1.2.0
# nltk>=3.7
# regex>=2022.10.31" > requirements.txt

# # Install all requirements at once:
# pip install -r requirements.txt

# # After installing spacy, download the language model:
# python -m spacy download en_core_web_lg

In [5]:
# Core Libraries
import os
import re
import string
import numpy as np
import pandas as pd

# PDF/DOCX Processing
import pdfplumber
from docx import Document

# NLP Processing
import spacy
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [6]:

# Bias Detection Parameters
BIAS_KEYWORDS = {
    'gender': ['he', 'she', 'mr', 'mrs', 'male', 'female', 'man', 'woman'],
    'race': ['asian', 'african', 'hispanic', 'caucasian'],
    'age': ['born', 'age', 'year']
}

# **2. Enhanced Resume Parsing**
### PDF Text Extraction with pdfplumber

In [7]:
def extract_text_from_pdf(file_path):
    """
    Extracts text from PDF while preserving layout using pdfplumber
    - Handles tables and bullet points better than PyPDF2
    - Returns cleaned text string
    """
    full_text = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            # Extract text with layout preservation
            text = page.extract_text(x_tolerance=1, y_tolerance=1)
            # Extract tables if any
            tables = page.extract_tables()
            for table in tables:
                text += "\n".join(["|".join(row) for row in table])
            full_text.append(text)
    return "\n".join(full_text)

### DOCX Text Extraction with Formatting


In [8]:
def extract_text_from_docx(file_path):
    """
    Extracts text from DOCX with paragraph separation
    - Preserves bullet points and section headers
    - Returns cleaned text string
    """
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        # Preserve bullet points and numbering
        if para.style.name.startswith('List'):
            full_text.append("• " + para.text)
        else:
            full_text.append(para.text)
    return "\n".join(full_text)

# **3. Text Preprocessing**

In [9]:
# def preprocess_text(text):
#     """
#     Cleans and normalizes text data:
#     1. Lowercase conversion
#     2. Remove punctuation
#     3. Remove numbers
#     4. Remove extra whitespace
#     """
#     # Convert to lowercase
#     text = text.lower()
    
#     # Remove punctuation
#     text = text.translate(str.maketrans('', '', string.punctuation))
    
#     # Remove numbers
#     text = re.sub(r'\d+', '', text)
    
#     # Remove extra whitespace
#     text = ' '.join(text.split())
    
#     return text

In [10]:
# ----------------------------
# 3. Improved Text Preprocessing
# ----------------------------
def preprocess_text(text):
    """
    Less aggressive preprocessing:
    - Keep numbers and some punctuation
    - Don't lowercase everything immediately
    """
    # Remove special characters except allowed ones
    text = re.sub(r'[^a-zA-Z0-9\s.,!?\-]', '', text)
    
    # Keep case but normalize whitespace
    text = ' '.join(text.split())
    
    return text


# **4. Word2Vec Implementation**

### Train Word2Vec Model


In [11]:
# def train_word2vec(resumes):
#     """
#     Trains Word2Vec model on resume corpus
#     - Converts resumes to tokenized sentences
#     - Creates 300-dimensional embeddings
#     """
#     # Tokenize resumes into sentences
#     tokenized_resumes = [re.split('[.!?]', resume) for resume in resumes]
    
#     # Flatten list of sentences
#     sentences = [sentence.strip().split() for resume in tokenized_resumes 
#                 for sentence in resume if sentence.strip()]
    
#     # Train Word2Vec model
#     model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)
#     return model

In [12]:
# ----------------------------
# 4. Enhanced Word2Vec Training
# ----------------------------
def train_word2vec(resumes):
    # Filter empty documents first
    non_empty_resumes = [resume for resume in resumes if len(resume.strip()) > 0]
    
    # Add fallback for empty corpus
    if len(non_empty_resumes) == 0:
        raise ValueError("No valid text documents found for training")
    
    tokenized_resumes = [re.split('[.!?]', resume) for resume in non_empty_resumes]
    sentences = [sentence.strip().split() for resume in tokenized_resumes 
                for sentence in resume if sentence.strip()]
    
    # Minimum 5 documents required for training
    if len(sentences) < 5:
        sentences += [['placeholder']] * (5 - len(sentences))
        
    model = Word2Vec(sentences, vector_size=100, window=3, min_count=1, workers=4)
    return model


### Document Embedding Generator


In [13]:
def get_doc_embedding(text, model):
    """
    Converts text to document embedding by:
    1. Tokenizing input text
    2. Averaging word vectors
    3. Handling out-of-vocabulary words
    """
    tokens = text.split()
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# **5. Bias Mitigation System**

### Bias Detection


In [14]:
def detect_bias(text):
    """
    Identifies potential biased language in text:
    - Returns dictionary with detected bias categories
    - Highlights specific keywords found
    """
    bias_report = {}
    text_lower = text.lower()
    
    for category, keywords in BIAS_KEYWORDS.items():
        found = [kw for kw in keywords if kw in text_lower]
        if found:
            bias_report[category] = found
            
    return bias_report

### Demographic Masking


In [15]:
def mask_demographics(text):
    """
    Removes personally identifiable information:
    1. Names (any capitalized words in sequence)
    2. Email addresses
    3. Phone numbers
    4. Physical addresses
    """
    # Mask names
    text = re.sub(r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b', '[NAME]', text)
    
    # Mask emails
    text = re.sub(r'\S+@\S+', '[EMAIL]', text)
    
    # Mask phone numbers
    text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[PHONE]', text)
    
    # Mask addresses
    text = re.sub(r'\d+\s+([a-zA-Z]+\s*)+', '[ADDRESS]', text)
    
    return text

# **6. Hybrid Ranking System**

### tf-idf AND Word2Vec

In [17]:
# ----------------------------
# 6. Updated Hybrid Ranking
# ----------------------------
def hybrid_ranking(job_desc, resumes, tfidf_weight=0.6, w2v_weight=0.4):
    # Filter empty resumes first
    valid_resumes = [resume for resume in resumes if len(resume.strip()) > 0]
    
    # Add fallback for empty input
    if len(valid_resumes) == 0:
        return np.array([]), np.array([])
    
    # Modified TF-IDF parameters
    tfidf_vectorizer = TfidfVectorizer(
        stop_words='english',
        min_df=2,  # Require words to appear in at least 2 documents
        max_df=0.8,  # Remove words appearing in >80% documents
        ngram_range=(1, 2)  # Include bigrams
    )
    
    try:
        tfidf_matrix = tfidf_vectorizer.fit_transform(valid_resumes)
    except ValueError:
        # Fallback to simple word presence
        tfidf_vectorizer = TfidfVectorizer(
            stop_words=None,
            min_df=1,
            token_pattern=r'(?u)\b\w+\b'
        )
        tfidf_matrix = tfidf_vectorizer.fit_transform(valid_resumes)
    
    # Rest of the function remains same...

# **7. Enhanced Feedback System**

### Recruiter Feedback


In [18]:
def generate_recruiter_feedback(resume, job_desc, vectorizer):
    """
    Generates detailed feedback for hiring managers:
    - Top matching skills
    - Missing required skills
    - Bias warnings
    """
    # TF-IDF analysis
    job_tfidf = vectorizer.transform([job_desc])
    resume_tfidf = vectorizer.transform([resume])
    
    # Get important keywords
    feature_names = vectorizer.get_feature_names_out()
    job_keywords = set(feature_names[job_tfidf.indices])
    resume_keywords = set(feature_names[resume_tfidf.indices])
    
    # Generate feedback components
    missing_skills = job_keywords - resume_keywords
    strengths = resume_keywords & job_keywords
    bias_report = detect_bias(resume)
    
    feedback = {
        'strengths': list(strengths)[:5],
        'missing_skills': list(missing_skills)[:5],
        'bias_warnings': bias_report,
        'score_breakdown': {
            'technical_skills': len(strengths),
            'missing_requirements': len(missing_skills)
        }
    }
    return feedback

### Job Seeker Feedback


In [19]:
def generate_applicant_feedback(resume, job_desc, vectorizer):
    """
    Generates actionable feedback for job seekers:
    - Suggested improvements
    - Missing keywords
    - Formatting tips
    """
    analysis = generate_recruiter_feedback(resume, job_desc, vectorizer)
    
    feedback = []
    if analysis['missing_skills']:
        feedback.append("Consider adding these skills: " + ", ".join(analysis['missing_skills']))
    if analysis['bias_warnings']:
        feedback.append("Potential bias detected: " + str(analysis['bias_warnings'].keys()))
    if not feedback:
        feedback.append("Strong match! Consider highlighting key achievements more prominently.")
    
    return "\n".join(feedback)

# **8. Full Pipeline Execution**

### # Step 1: Load Resumes

In [20]:
# def load_resumes(folder_path):
#     resumes = []
#     for file_name in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, file_name)
#         if file_name.endswith(".pdf"):
#             text = extract_text_from_pdf(file_path)
#         elif file_name.endswith(".docx"):
#             text = extract_text_from_docx(file_path)
#         else:
#             continue
#         # Apply preprocessing and masking
#         cleaned_text = preprocess_text(mask_demographics(text))
#         resumes.append(cleaned_text)
#     return resumes

### # Step 2: Define Job Description


In [21]:
# job_description = """
# Looking for software engineer with 3+ years experience in Python, machine learning, 
# and cloud technologies. Strong background in NLP and data analysis preferred.
# """

### # Step 3: Process and Rank


In [22]:
# resumes = load_resumes("Datasets/data")
# ranked_indices, scores = hybrid_ranking(job_description, resumes)

In [23]:
# # After loading resumes
# print(f"Loaded {len(resumes)} resumes")
# print("Sample resume text:", resumes[0][:200] if len(resumes) > 0 else "No resumes")

In [24]:
# # Before TF-IDF transformation
# print("Vocabulary size:", len(tfidf_vectorizer.vocabulary_))

### # Step 4: Generate Reports


In [25]:
# vectorizer = TfidfVectorizer(stop_words='english').fit(resumes + [job_description])
# final_results = []

# for idx in ranked_indices:
#     resume = resumes[idx]
#     final_results.append({
#         'rank': len(final_results)+1,
#         'score': scores[idx],
#         'recruiter_feedback': generate_recruiter_feedback(resume, job_description, vectorizer),
#         'applicant_feedback': generate_applicant_feedback(resume, job_description, vectorizer)
#     })


### # Convert to DataFrame


In [26]:
# results_df = pd.DataFrame(final_results)
# results_df.to_csv("ranked_results.csv", index=False)

## **Key Improvements Added:**

Advanced PDF parsing with layout preservation

Word2Vec semantic understanding

Bias detection and demographic masking

Hybrid ranking system (TF-IDF + Word2Vec)

Dual feedback system (recruiter + applicant views)

Comprehensive text preprocessing pipeline

In [27]:
# # Step 1: Load Resumes
# def load_resumes(folder_path):
#     resumes = []
#     for file_name in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, file_name)
#         if file_name.endswith(".pdf"):
#             text = extract_text_from_pdf(file_path)
#         elif file_name.endswith(".docx"):
#             text = extract_text_from_docx(file_path)
#         else:
#             continue
#         # Apply preprocessing and masking
#         cleaned_text = preprocess_text(mask_demographics(text))
#         resumes.append(cleaned_text)
#     return resumes

# # Step 2: Define Job Description
# job_description = """
# Looking for software engineer with 3+ years experience in Python, machine learning, 
# and cloud technologies. Strong background in NLP and data analysis preferred.
# """

# # Step 3: Process and Rank
# resumes = load_resumes("Datasets/data")
# ranked_indices, scores = hybrid_ranking(job_description, resumes)

# # Step 4: Generate Reports
# vectorizer = TfidfVectorizer(stop_words='english').fit(resumes + [job_description])
# final_results = []

# for idx in ranked_indices:
#     resume = resumes[idx]
#     final_results.append({
#         'rank': len(final_results)+1,
#         'score': scores[idx],
#         'recruiter_feedback': generate_recruiter_feedback(resume, job_description, vectorizer),
#         'applicant_feedback': generate_applicant_feedback(resume, job_description, vectorizer)
#     })

# # Convert to DataFrame
# results_df = pd.DataFrame(final_results)
# results_df.to_csv("ranked_results.csv", index=False)

In [28]:
# ----------------------------
# 1. Revised Resume Loading for Nested Folders
# ----------------------------
def load_resumes(root_folder):
    """
    Loads resumes from nested folder structure:
    Project/Datasets/data/data/{role_subfolders}/
    """
    resumes = []
    total_files = 0
    
    # Corrected base path
    base_path = os.path.join(root_folder, "data")  # Now goes into Project/Datasets/data/data/
    
    print(f"Searching in: {os.path.abspath(base_path)}")
    
    for dirpath, _, filenames in os.walk(base_path):
        for filename in filenames:
            # Case-insensitive check
            if filename.lower().endswith((".pdf", ".docx")):
                total_files += 1
                file_path = os.path.join(dirpath, filename)
                try:
                    if filename.endswith(".pdf"):
                        text = extract_text_from_pdf(file_path)
                    else:
                        text = extract_text_from_docx(file_path)
                        
                    cleaned_text = preprocess_text(mask_demographics(text))
                    resumes.append(cleaned_text)
                    print(f"✓ Processed: {filename}")
                    
                except Exception as e:
                    print(f"✗ Error in {filename}: {str(e)}")
    
    print(f"\nTotal resume files found: {total_files}")
    print(f"Successfully processed: {len(resumes)}")
    return resumes

# ----------------------------
# 2. How to Use It
# ----------------------------
# Path should point to: Project/Datasets/data/
dataset_path = "Datasets/data/"

# The function will automatically go into the nested /data/ folder
resumes = load_resumes(dataset_path)


Searching in: F:\CSE499A,B\New 499B\Datasets\data\data
✓ Processed: 10554236.pdf
✓ Processed: 10674770.pdf
✓ Processed: 11163645.pdf
✓ Processed: 11759079.pdf
✓ Processed: 12065211.pdf
✓ Processed: 12202337.pdf
✓ Processed: 12338274.pdf
✓ Processed: 12442909.pdf
✓ Processed: 12780508.pdf
✓ Processed: 12802330.pdf
✓ Processed: 13072019.pdf
✓ Processed: 13130984.pdf
✓ Processed: 13294301.pdf
✓ Processed: 13491889.pdf
✓ Processed: 13701259.pdf
✓ Processed: 14055988.pdf
✓ Processed: 14126433.pdf
✓ Processed: 14224370.pdf
✓ Processed: 14449423.pdf
✓ Processed: 14470533.pdf
✓ Processed: 14491649.pdf
✓ Processed: 14496667.pdf
✓ Processed: 15289348.pdf
✓ Processed: 15363277.pdf
✓ Processed: 15592167.pdf
✓ Processed: 15821633.pdf
✓ Processed: 15906625.pdf
✓ Processed: 16237710.pdf
✓ Processed: 17306905.pdf
✓ Processed: 17407184.pdf
✓ Processed: 17556527.pdf
✓ Processed: 18132924.pdf
✓ Processed: 18365791.pdf
✓ Processed: 18569929.pdf
✓ Processed: 18635654.pdf
✓ Processed: 18669563.pdf
✓ Process

In [29]:
# Step 2: Job Description with Validation
job_description = """
Looking for software engineer with 3+ years experience in Python, machine learning, 
and cloud technologies. Strong background in NLP and data analysis preferred.
"""

if not job_description.strip():
    raise ValueError("Job description cannot be empty!")

# Step 3: Process and Rank with Checks
print("\n=== Starting Processing ===")
resumes = load_resumes("Datasets/data")

if len(resumes) == 0:
    raise ValueError("No resumes loaded! Check input files and folder structure.")

try:
    ranked_indices, scores = hybrid_ranking(job_description, resumes)
    print(f"Ranking completed. Found {len(ranked_indices)} results")
except Exception as e:
    print(f"Ranking failed: {str(e)}")
    ranked_indices, scores = [], []

# Step 4: Enhanced Reporting with Validation
print("\n=== Generating Reports ===")
if len(ranked_indices) == 0:
    print("Warning: No rankings generated. Creating empty report.")
    final_results = []
else:
    try:
        vectorizer = TfidfVectorizer(stop_words='english').fit(resumes + [job_description])
        final_results = []
        
        for idx in ranked_indices:
            if idx >= len(resumes):
                print(f"Invalid index {idx} skipped")
                continue
                
            resume = resumes[idx]
            print(f"Processing rank {len(final_results)+1} (index {idx})")
            
            final_results.append({
                'rank': len(final_results)+1,
                'score': scores[idx],
                'resume_preview': resume[:500] + "...",
                'recruiter_feedback': generate_recruiter_feedback(resume, job_description, vectorizer),
                'applicant_feedback': generate_applicant_feedback(resume, job_description, vectorizer)
            })
            
    except Exception as e:
        print(f"Report generation failed: {str(e)}")
        final_results = []

# Step 5: Safe CSV Export with Preview
print("\n=== Final Output ===")
if len(final_results) == 0:
    print("Warning: No results to export. Creating empty CSV.")
    results_df = pd.DataFrame(columns=['rank', 'score', 'resume_preview', 
                                      'recruiter_feedback', 'applicant_feedback'])
else:
    results_df = pd.DataFrame(final_results)
    print("Preview of first 3 results:")
    print(results_df.head(3).to_string())

results_df.to_csv("ranked_results.csv", index=False)
print("\n=== CSV file saved successfully ===")


=== Starting Processing ===
Searching in: F:\CSE499A,B\New 499B\Datasets\data\data
✓ Processed: 10554236.pdf
✓ Processed: 10674770.pdf
✓ Processed: 11163645.pdf
✓ Processed: 11759079.pdf
✓ Processed: 12065211.pdf
✓ Processed: 12202337.pdf
✓ Processed: 12338274.pdf
✓ Processed: 12442909.pdf
✓ Processed: 12780508.pdf
✓ Processed: 12802330.pdf
✓ Processed: 13072019.pdf
✓ Processed: 13130984.pdf
✓ Processed: 13294301.pdf
✓ Processed: 13491889.pdf
✓ Processed: 13701259.pdf
✓ Processed: 14055988.pdf
✓ Processed: 14126433.pdf
✓ Processed: 14224370.pdf
✓ Processed: 14449423.pdf
✓ Processed: 14470533.pdf
✓ Processed: 14491649.pdf
✓ Processed: 14496667.pdf
✓ Processed: 15289348.pdf
✓ Processed: 15363277.pdf
✓ Processed: 15592167.pdf
✓ Processed: 15821633.pdf
✓ Processed: 15906625.pdf
✓ Processed: 16237710.pdf
✓ Processed: 17306905.pdf
✓ Processed: 17407184.pdf
✓ Processed: 17556527.pdf
✓ Processed: 18132924.pdf
✓ Processed: 18365791.pdf
✓ Processed: 18569929.pdf
✓ Processed: 18635654.pdf
✓ Proc

In [30]:
# Import libraries
import os
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Configuration
DATASET_PATH = "Datasets/data/"  # Path to your main data folder
JOB_DESCRIPTION = """
Looking for software engineer with 3+ years experience in Python, machine learning, 
and cloud technologies. Strong background in NLP and data analysis preferred.
"""

In [31]:
def load_resumes_silently(root_folder):
    """Load resumes without per-file output"""
    resumes = []
    total_files = 0
    error_count = 0
    
    # Nested folder structure handling
    base_path = os.path.join(root_folder, "data")
    
    # Progress tracking
    print(f"⏳ Scanning resumes in: {os.path.abspath(base_path)}")
    
    for dirpath, _, filenames in os.walk(base_path):
        for filename in filenames:
            if filename.lower().endswith((".pdf", ".docx")):
                total_files += 1
                file_path = os.path.join(dirpath, filename)
                try:
                    # Process every 100 files to show progress
                    if total_files % 100 == 0:
                        print(f"📁 Processed {total_files} files...")
                        
                    # Your existing processing logic
                    if filename.endswith(".pdf"):
                        text = extract_text_from_pdf(file_path)
                    else:
                        text = extract_text_from_docx(file_path)
                        
                    cleaned_text = preprocess_text(mask_demographics(text))
                    resumes.append(cleaned_text)
                    
                except Exception as e:
                    error_count += 1

    # Final report
    print(f"\n✅ Completed!\nTotal files: {total_files}\nSuccess: {len(resumes)}\nErrors: {error_count}")
    return resumes

In [32]:
# Load resumes with minimal output
print("=== STARTING PROCESS ===")
resumes = load_resumes_silently(DATASET_PATH)

# Check resume loading
if not resumes:
    raise ValueError("No resumes loaded! Check dataset path and file formats.")

# Run hybrid ranking
try:
    print("\n⚙️ Analyzing resumes...")
    ranked_indices, scores = hybrid_ranking(JOB_DESCRIPTION, resumes)
    
    if len(ranked_indices) == 0:
        print("⚠️ No matching candidates found!")
    else:
        print(f"🎯 Found {len(ranked_indices)} potential candidates")
        
except Exception as e:
    print(f"❌ Ranking failed: {str(e)}")
    ranked_indices, scores = [], []

=== STARTING PROCESS ===
⏳ Scanning resumes in: F:\CSE499A,B\New 499B\Datasets\data\data
📁 Processed 100 files...
📁 Processed 200 files...
📁 Processed 300 files...
📁 Processed 400 files...
📁 Processed 500 files...
📁 Processed 600 files...
📁 Processed 700 files...
📁 Processed 800 files...
📁 Processed 900 files...
📁 Processed 1000 files...
📁 Processed 1100 files...
📁 Processed 1200 files...
📁 Processed 1300 files...
📁 Processed 1400 files...
📁 Processed 1500 files...
📁 Processed 1600 files...
📁 Processed 1700 files...
📁 Processed 1800 files...
📁 Processed 1900 files...
📁 Processed 2000 files...
📁 Processed 2100 files...
📁 Processed 2200 files...
📁 Processed 2300 files...
📁 Processed 2400 files...

✅ Completed!
Total files: 2484
Success: 2483
Errors: 1

⚙️ Analyzing resumes...
❌ Ranking failed: cannot unpack non-iterable NoneType object


In [33]:
def generate_results(resumes, job_desc, ranked_indices, scores):
    """Generate DataFrame without saving"""
    results = []
    
    if not ranked_indices:
        return pd.DataFrame()
    
    vectorizer = TfidfVectorizer(stop_words='english').fit(resumes + [job_desc])
    
    for rank, idx in enumerate(ranked_indices[:100]):  # Only top 100
        resume_text = resumes[idx]
        results.append({
            'Rank': rank + 1,
            'Resume Index': idx + 1,
            'Resume Preview': resume_text[:500] + "...",
            'Score': scores[idx],
            'Recruiter Feedback': generate_recruiter_feedback(resume_text, job_desc, vectorizer),
            'Applicant Feedback': generate_applicant_feedback(resume_text, job_desc, vectorizer)
        })
    
    return pd.DataFrame(results)

# Generate and display results
print("\n📊 Generating results...")
results_df = generate_results(resumes, JOB_DESCRIPTION, ranked_indices, scores)

if not results_df.empty:
    print("\nTop 5 Candidates:")
    print(results_df.head(5).to_string(index=False))
else:
    print("No results to display")


📊 Generating results...
No results to display


In [35]:
results_df.head()

""
